https://keras.io/examples/lstm_seq2seq/

## Sequence to sequence example in Keras (character-level).

This script demonstrates how to implement a basic character-level sequence-to-sequence model. We apply it to translating short English sentences into short French sentences, character-by-character. Note that it is fairly unusual to do character-level machine translation, as word-level models are more common in this domain.

In [9]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import random

In [10]:
path = 'phr1.txt'
text = open(path, encoding='utf-8').read()
print('Corpus length:', len(text))

Corpus length: 64974


In [11]:
raw_lines = text.splitlines()
lines = []
for i in range(0, len(raw_lines), 3):
    inp = raw_lines[i].strip()
    outp = raw_lines[i+1].strip()
    lines.append(f"{inp}\t{outp}")
    
random.shuffle(lines)

print("Pairs:", len(lines))
print(lines[:10])

Pairs: 865
['What are you working on?\tI am baking a cake.', "you are disgusting\tthat's okay.  disgusting is good.", 'you are a paranoid\tyou are a paranoid', 'who wrote the hobbit\tThe Hobbit was written by J.R.R. Tolkein.', 'Hi, How is it going?\tFine', "You sound like Data\tYes I am inspired by commander Data's artificial personality.", 'do androids dream of electric sheep\tThat is a good story by Philip K. Dick.  Have you read VALIS or The Man In the High Castle?', 'Tell me a joke\t\'two vultures boarded a plane, each carrying two dead raccoons. the  stewardess stops them and says "sorry sir, only one carrion per  passenger." \'', "There are other entities who are better versed on the topic.\tI don't think I have much to say on the topic.", 'Something fun\tBots are a lot of fun..']


In [12]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

In [13]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
#with open(data_path, 'r', encoding='utf-8') as f:
#    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 864
Number of unique input tokens: 70
Number of unique output tokens: 77
Max sequence length for inputs: 182
Max sequence length for outputs: 218


In [14]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [19]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 691 samples, validate on 173 samples
Epoch 1/10
691/691 [==============================] - 9s 13ms/step - loss: 0.7325 - val_loss: 0.7112
Epoch 2/10
691/691 [==============================] - 7s 11ms/step - loss: 0.6499 - val_loss: 0.7059
Epoch 3/10
691/691 [==============================] - 9s 12ms/step - loss: 0.6445 - val_loss: 0.7035
Epoch 4/10
691/691 [==============================] - 9s 13ms/step - loss: 0.6411 - val_loss: 0.6973
Epoch 5/10
691/691 [==============================] - 10s 14ms/step - loss: 0.6375 - val_loss: 0.6928
Epoch 6/10
691/691 [==============================] - 9s 13ms/step - loss: 0.6332 - val_loss: 0.6860
Epoch 7/10
691/691 [==============================] - 9s 14ms/step - loss: 0.6262 - val_loss: 0.6852
Epoch 8/10
691/691 [==============================] - 9s 13ms/step - loss: 0.6208 - val_loss: 0.6743
Epoch 9/10
691/691 [==============================] - 9s 13ms/step - loss: 0.6137 - val_loss: 0.6702
Epoch 10/10
691/691 [=======================

C:\Users\ssotn\Anaconda3\envs\nlp\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_6 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_5/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_5/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [20]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [21]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: What are you working on?
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: you are disgusting
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: you are a paranoid
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: who wrote the hobbit
Decoded sentence: I                                                                                                               

-
Input sentence: A curious question, to be sure.
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: what is government
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: what is the stock market
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: Who is your appointent with?
Decoded sentence: I                                                                                          

-
Input sentence: you are hopeless
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: What is a chat robot?
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: Tell me a joke
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: What are your hobbies?
Decoded sentence: I                                                                                                                      

-
Input sentence: have you read plato
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: william gibson
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: Can you move
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: If you could eat food, what would you eat?
Decoded sentence: I                                                                                                        

In [22]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[-seq_index-2: -seq_index-1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[-seq_index-2])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: chemistry
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: you are not smart
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: Tell me a joke
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: What have I done that isn't nice?
Decoded sentence: I                                                                                                                      

-
Input sentence: Is it cramped in the computer
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: ARE YOU A FOOTBALL
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: WHAT SOCCER
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: Who are you?
Decoded sentence: I                                                                                                                         

-
Input sentence: Although never is often better than right now.
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: Can you breathe
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: you are crazy
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: Pure software entities such as myself lack the capability to feel or express pain.
Decoded sentence: I                                   

-
Input sentence: What was the name of the first artificial Earth satellite?
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: did tell gossips to anybody
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: How are you doing?
Decoded sentence: I                                                                                                                                                                                                                          
-
Input sentence: What's up?
Decoded sentence: I                                                                              